In [29]:
import pandas as pd
import numpy as np
import re


def natural_sort(l):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)


root_folder = 'drive/MyDrive/CARD0003/2024/'
# root_folder = './'
formatted_reponses = pd.read_csv(f'{root_folder}/reference/formatted_reponses.csv', index_col = 0)
questions = list(formatted_reponses.columns)
question_selection = questions
formatted_reponses = formatted_reponses[questions]
candidates = formatted_reponses.drop('score').index
scores = formatted_reponses.loc['score']
responses = formatted_reponses.drop('score')

sum_scores = np.sum([float(score) for score in scores])

In [30]:
def get_question_selection(initials):
  marking_groups = 'Marking Groups.xlsx'
  marking_groups = pd.read_excel(f'{root_folder}/reference/{marking_groups}').iloc[2:]
  marking_groups.columns = ['Q','mark','marker1','marker2']

  question_selection = list(marking_groups.loc[(marking_groups['marker1'] == initials) | (marking_groups['marker2'] == initials)].Q.values)
  return question_selection

In [31]:
def sum_question(initials):
    marks = pd.read_csv(f'{root_folder}/results/scores_{initials}.csv').set_index('ID').astype('int')
    questions = get_question_selection(initials)
    collated = {}
    for i in questions:
        current_question = [col for col in list(marks.columns) if f"Q{i}." in col]
        collated[f'Question {i}'] = marks[current_question].sum(axis =1 )
    return pd.DataFrame(collated).replace(0,np.nan).dropna(axis = 1, how = 'all').replace(np.nan, 0)

In [32]:
def collate(initials1,initials2):
    marks_1 = sum_question(initials1)
    marks_2 = sum_question(initials2)

    marks_1_2  = marks_1.merge(marks_2, on = 'ID', suffixes = (f'_1',f'_2')).T.sort_index().T

    questions = marks_1.columns
    for question in questions:
        marks_1_2[f"{question}_Average"] =  marks_1_2[[f"{question}_1",f"{question}_2"]].mean(axis = 1)
    marks_1_2 = marks_1_2.T.sort_index()

    new_index = pd.MultiIndex.from_arrays(zip(*marks_1_2.index.str.split('_')))

    marks_1_2.index = new_index
    marks_1_2 = marks_1_2.T
    for col in marks_1_2.columns:
        if 'Average' not in col:
            marks_1_2[col] = marks_1_2[col].astype('int')
    return marks_1_2


In [33]:
def combine_marks(marks1, marks2):
    df = pd.concat([marks1, marks2], axis = 1)
    # Extract column levels and sort them
    column_levels = df.columns.levels[0]
    sorted_column_levels = natural_sort(column_levels)

    # Reorder the columns according to sorted column levels
    df = df.reindex(columns=pd.MultiIndex.from_product([sorted_column_levels, df.columns.levels[1]]))
    return df

def add_total(df):
    total_1 = df.loc[:, (slice(None), '1')].sum(axis=1)
    total_2 = df.loc[:, (slice(None), '2')].sum(axis=1)

    # Calculate the average between subheadings '1' and '2'
    average_between_1_and_2 = (total_1 + total_2) / 2

    # Concatenate totals and average to the DataFrame
    totals_and_average = pd.concat([total_1, total_2, average_between_1_and_2], axis=1)
    totals_and_average.columns = [('Total', '1'), ('Total', '2'), ('Total', 'Average')]

    # Combine totals and average with the original DataFrame
    df = pd.concat([df, totals_and_average], axis=1)
    return df

def add_percent(df):
    percent_1 = round(df[('Total', '1')]*100/sum_scores, 1)
    percent_2 = round(df[('Total', '2')]*100/sum_scores, 1)

    average_percent = round(df[('Total', 'Average')]*100/sum_scores, 1)


    # Concatenate totals and average to the DataFrame
    percentages = pd.concat([percent_1, percent_2, average_percent], axis=1)
    percentages.columns = [('Grade', '1'), ('Grade', '2'), ('Grade', 'Average')]

    # Combine totals and average with the original DataFrame
    df = pd.concat([df, percentages], axis=1)
    return df


In [34]:
marks_CL_VM = collate(initials1 = 'CL',initials2 = 'VM')
marks_AC_AM = collate(initials1 = 'AC',initials2 = 'AM')

In [35]:
df = combine_marks(marks_CL_VM, marks_AC_AM)
df = add_total(df)
df = add_percent(df)

In [36]:
candidates = pd.read_excel(f'{root_folder}/reference/Case-Based_Answer.xlsx').iloc[:,[0,8]]
candidates = candidates.rename(columns={candidates.columns[1]:'Candidate_Number'}).set_index('ID').iloc[:,0]
candidate_dict = candidates.to_dict()

In [37]:
df['Candidate_Number'] = df.index.map(candidate_dict)
df = df.reset_index().set_index(['ID','Candidate_Number'])


In [38]:
df.to_excel('CARD0003_2024_marks.xlsx')